In [2]:
import os
import math
import shutil
import numpy as np
from multiprocessing import Pool
from keras.optimizers import RMSprop, SGD
from keras.losses import categorical_crossentropy
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

Using TensorFlow backend.


In [32]:
GPU = '0'
RND = 1
RUN = 'A'
OUT_DIR = 'out'
TENSORBOARD_DIR = '/tensorboard/tf-speech-v2/%s' % RUN
MODELS_DIR = '%s/models/run_%s/fold_$fold$' % (OUT_DIR, RUN)
INPUT_SIZE = (64, 64, 1)  # n_mels x width x 1ch
FOLDS = 10

In [45]:
%run '../data-generator.ipynb'

In [33]:
fold = 0

In [49]:
val_X = np.load('%s/val/val_X_%d.npy' % (OUT_DIR, fold))
val_Y = np.load('%s/val/val_Y_%d.npy' % (OUT_DIR, fold))
val_files = np.load('%s/val/val_files_%d.npy' % (OUT_DIR, fold))

assert len(val_X) == len(val_files)
assert len(val_Y) == len(val_files)

print('len(val_X):', len(val_X))

len(val_X): 2441


In [39]:
models_dir = MODELS_DIR.replace('$fold$', str(fold))
os.makedirs(models_dir, exist_ok=True)

In [51]:
train_X = np.memmap('%s/train_X.mem' % OUT_DIR, np.float32,
                    'r').reshape((-1, ) + INPUT_SIZE)
train_Y = np.memmap('%s/train_Y.mem' % OUT_DIR, np.float32, 'r').reshape(
    (-1, len(LABELS)))

train_files = np.load('%s/train_files.npy' % OUT_DIR)

assert len(train_Y) == len(train_X)
assert len(train_files) == len(train_X)

In [52]:
train_files

array(['yes/e1469561_nohash_3.wav', 'go/012c8314_nohash_0.wav',
       'up/6f342826_nohash_1.wav', ..., 'stop/e04d7130_nohash_2.wav',
       'up/b7a0754f_nohash_0.wav', 'go/3209ec42_nohash_2.wav'], dtype=object)